In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# Only use if using Google Drive
!pip install chess
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/deep-learning-su2024-chess-project/

import HelperFunc

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/deep-learning-su2024-chess-project


In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe):
        self.df = pd.read_csv(dataframe)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df['Human?'].iloc[idx]
        bitboard = HelperFunc.fen_to_bitboard(self.df['FEN'].iloc[idx], self.df['Moves'].iloc[idx])
        return bitboard, label

In [5]:
data_df = pd.read_csv('data.csv')

In [32]:
batch_size = 40
num_iters = 16000
num_epochs = num_iters / (len(data_df) / batch_size)
num_epochs = int(num_epochs)

In [7]:
train_dataloader = DataLoader(CustomImageDataset('data.csv'), batch_size=256, shuffle=True)
test_dataloader = DataLoader(CustomImageDataset('data.csv'), batch_size=256, shuffle=True)

In [39]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # Non-linearity 1
        self.relu1 = nn.ReLU()

        # Linear function 2
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        self.relu2 = nn.ReLU()

        # Linear function 3
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        self.relu3 = nn.ReLU()

        # Linear function 4 (readout)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)

        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.relu3(out)

        # Linear function 4 (readout)
        out = self.fc4(out)
        return out

In [9]:
input_dim = 517
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [10]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [41]:
iter = 0
for epoch in range(num_epochs):
    for i, (bitboard, labels) in enumerate(train_dataloader):
        bitboard = bitboard.to(torch.float32)
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(bitboard)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for bitboard, labels in test_dataloader:
                bitboard = bitboard.to(torch.float32)
                # Forward pass only to get logits/output
                outputs = model(bitboard)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.4129773676395416. Accuracy: 80.60244750976562
Iteration: 1000. Loss: 0.3360779583454132. Accuracy: 82.68936157226562
Iteration: 1500. Loss: 0.45257368683815. Accuracy: 81.48062133789062
Iteration: 2000. Loss: 0.25054919719696045. Accuracy: 88.27238464355469
